# LangSmith Walkthrough

LangChain makes it easy to prototype LLM applications and Agents. However, delivering LLM applications to production can be deceptively difficult. You will have to iterate on your prompts, chains, and other components to build a high-quality product.

LangSmith makes it easy to debug, test, and continuously improve your LLM applications.

When might this come in handy? You may find it useful when you want to:

- Quickly debug a new chain, agent, or set of tools
- Create and manage datasets for fine-tuning, few-shot prompting, and evaluation
- Run regression tests on your application to confidently develop
- Capture production analytics for product insights and continuous improvements

## Prerequisites

**[Create a LangSmith account](https://smith.langchain.com/) and create an API key (see bottom left corner). Familiarize yourself with the platform by looking through the [docs](https://docs.smith.langchain.com/)**

Note LangSmith is in closed beta; we're in the process of rolling it out to more users. However, you can fill out the form on the website for expedited access.

Now, let's get started!

## Log runs to LangSmith

First, configure your environment variables to tell LangChain to log traces. This is done by setting the `LANGCHAIN_TRACING_V2` environment variable to true.
You can tell LangChain which project to log to by setting the `LANGCHAIN_PROJECT` environment variable (if this isn't set, runs will be logged to the `default` project). This will automatically create the project for you if it doesn't exist. You must also set the `LANGCHAIN_ENDPOINT` and `LANGCHAIN_API_KEY` environment variables.

For more information on other ways to set up tracing, please reference the [LangSmith documentation](https://docs.smith.langchain.com/docs/).

**NOTE:** You can also use a context manager in python to log traces using

```python
from langchain_core.tracers.context import tracing_v2_enabled

with tracing_v2_enabled(project_name="My Project"):
    agent.run("How many people live in canada as of 2023?")
```

However, in this example, we will use environment variables.

In [2]:
import os
from uuid import uuid4
from dotenv import load_dotenv

load_dotenv()  # take environment variables from .env.

unique_id = uuid4().hex[0:8]
os.environ["LANGCHAIN_TRACING_V2"] = "true"
os.environ["LANGCHAIN_PROJECT"] = f"Tracing Walkthrough - {unique_id}"
os.environ["LANGCHAIN_ENDPOINT"] = "https://api.smith.langchain.com"

Create the langsmith client to interact with the API

In [3]:
from langsmith import Client

client = Client()

Create a LangChain component and log runs to the platform. In this example, we will create a ReAct-style agent with access to a general search tool (DuckDuckGo). The agent's prompt can be viewed in the [Hub here](https://smith.langchain.com/hub/wfh/langsmith-agent-prompt).

In [4]:
from langchain import hub
from langchain.agents import AgentExecutor
from langchain.agents.format_scratchpad.openai_tools import (
    format_to_openai_tool_messages,
)
from langchain.agents.output_parsers.openai_tools import OpenAIToolsAgentOutputParser
from langchain_community.tools import DuckDuckGoSearchResults
from langchain_openai import ChatOpenAI

# Fetches the latest version of this prompt
prompt = hub.pull("wfh/langsmith-agent-prompt:5d466cbc")

llm = ChatOpenAI(
    model="gpt-3.5-turbo",
    temperature=0,
)

tools = [
    DuckDuckGoSearchResults(
        name="duck_duck_go"
    ),  # General internet search using DuckDuckGo
]

llm_with_tools = llm.bind_tools(tools)

runnable_agent = (
    {
        "input": lambda x: x["input"],
        "agent_scratchpad": lambda x: format_to_openai_tool_messages(
            x["intermediate_steps"]
        ),
    }
    | prompt
    | llm_with_tools
    | OpenAIToolsAgentOutputParser()
)

agent_executor = AgentExecutor(
    agent=runnable_agent, tools=tools, handle_parsing_errors=True
)

We are running the agent concurrently on multiple inputs to reduce latency. Runs get logged to LangSmith in the background so execution latency is unaffected.

In [5]:
inputs = [
    "What is LangChain?",
    "What's LangSmith?",
    "When was Llama-v2 released?",
    "What is the langsmith cookbook?",
    "When did langchain first announce the hub?",
]

results = agent_executor.batch([{"input": x} for x in inputs], return_exceptions=True)

In [6]:
results[:2]

[{'input': 'What is LangChain?',
  'output': 'LangChain is a model-agnostic, open-source project that helps AI developers integrate large language models with various external data sources. It simplifies the creation of applications using large language models by providing a standard interface for chains, integrations with other tools, and end-to-end chains for common applications. LangChain allows developers to build applications based on combined Large Language Models. You can learn more about LangChain from sources like [Built In](https://builtin.com/articles/langchain), [GeeksforGeeks](https://www.geeksforgeeks.org/introduction-to-langchain/), [PyPI](https://pypi.org/project/langchain/), and [Nanonets](https://nanonets.com/blog/langchain/).'},
 {'input': "What's LangSmith?",
  'output': 'LangSmith is a platform that allows you to test and evaluate the capabilities of language models and AI applications. It helps in building chat-based AI environments with LangChain, creating LangSm

Assuming you've successfully set up your environment, your agent traces should show up in the `Projects` section in the [app](https://smith.langchain.com/). Congrats!

![Initial Runs](https://github.com/langchain-ai/langchain/blob/master/docs/docs/langsmith/img/log_traces.png?raw=1)

It looks like the agent isn't effectively using the tools though. Let's evaluate this so we have a baseline.

## Evaluate Agent

In addition to logging runs, LangSmith also allows you to test and evaluate your LLM applications.

In this section, you will leverage LangSmith to create a benchmark dataset and run AI-assisted evaluators on an agent. You will do so in a few steps:

1. Create a dataset
2. Initialize a new agent to benchmark
3. Configure evaluators to grade an agent's output
4. Run the agent over the dataset and evaluate the results

### 1. Create a LangSmith dataset

Below, we use the LangSmith client to create a dataset from the input questions from above and a list labels. You will use these later to measure performance for a new agent. A dataset is a collection of examples, which are nothing more than input-output pairs you can use as test cases to your application.

For more information on datasets, including how to create them from CSVs or other files or how to create them in the platform, please refer to the [LangSmith documentation](https://docs.smith.langchain.com/).

In [7]:
outputs = [
    "LangChain is an open-source framework for building applications using large language models. It is also the name of the company building LangSmith.",
    "LangSmith is a unified platform for debugging, testing, and monitoring language model applications and agents powered by LangChain",
    "July 18, 2023",
    "The langsmith cookbook is a github repository containing detailed examples of how to use LangSmith to debug, evaluate, and monitor large language model-powered applications.",
    "September 5, 2023",
]

In [8]:
dataset_name = f"agent-qa-{unique_id}"

dataset = client.create_dataset(
    dataset_name,
    description="An example dataset of questions over the LangSmith documentation.",
)

client.create_examples(
    inputs=[{"input": query} for query in inputs],
    outputs=[{"output": answer} for answer in outputs],
    dataset_id=dataset.id,
)

### 2. Initialize a new agent to benchmark

LangSmith lets you evaluate any LLM, chain, agent, or even a custom function. Conversational agents are stateful (they have memory); to ensure that this state isn't shared between dataset runs, we will pass in a `chain_factory` (aka a `constructor`) function to initialize for each call.

In this case, we will test an agent that uses OpenAI's function calling endpoints.

In [9]:
from langchain import hub
from langchain.agents import AgentExecutor, AgentType, initialize_agent, load_tools
from langchain_openai import ChatOpenAI


# Since chains can be stateful (e.g. they can have memory), we provide
# a way to initialize a new chain for each row in the dataset. This is done
# by passing in a factory function that returns a new chain for each row.
def create_agent(prompt, llm_with_tools):
    runnable_agent = (
        {
            "input": lambda x: x["input"],
            "agent_scratchpad": lambda x: format_to_openai_tool_messages(
                x["intermediate_steps"]
            ),
        }
        | prompt
        | llm_with_tools
        | OpenAIToolsAgentOutputParser()
    )
    return AgentExecutor(agent=runnable_agent, tools=tools, handle_parsing_errors=True)

### 3. Configure evaluation

Manually comparing the results of chains in the UI is effective, but it can be time consuming.
It can be helpful to use automated metrics and AI-assisted feedback to evaluate your component's performance.

Below, we will create a custom run evaluator that logs a heuristic evaluation.

**Heuristic evaluators**

In [10]:
from langsmith.evaluation import EvaluationResult
from langsmith.schemas import Example, Run


def check_not_idk(run: Run, example: Example):
    """Illustration of a custom evaluator."""
    agent_response = run.outputs["output"]
    if "don't know" in agent_response or "not sure" in agent_response:
        score = 0
    else:
        score = 1
    # You can access the dataset labels in example.outputs[key]
    # You can also access the model inputs in run.inputs[key]
    return EvaluationResult(
        key="not_uncertain",
        score=score,
    )

#### Batch Evaluators

Some metrics are aggregated over a full "test" without being assigned to an individual runs/examples. These could be as simple
as common classification metrics like Precision, Recall, or AUC, or it could be another custom aggregate metric.

You can define any batch metric on a full test level by defining a function (or any callable) that accepts a list of Runs (system traces) and list of Examples (dataset records).

In [11]:
from typing import List


def max_pred_length(runs: List[Run], examples: List[Example]):
    predictions = [len(run.outputs["output"]) for run in runs]
    return EvaluationResult(key="max_pred_length", score=max(predictions))

Below, we will configure the evaluation with the custom evaluator from above, as well as some pre-implemented run evaluators that do the following:
- Compare results against ground truth labels.
- Measure semantic (dis)similarity using embedding distance
- Evaluate 'aspects' of the agent's response in a reference-free manner using custom criteria

For a longer discussion of how to select an appropriate evaluator for your use case and how to create your own
custom evaluators, please refer to the [LangSmith documentation](https://docs.smith.langchain.com/).

In [12]:
from langchain.evaluation import EvaluatorType
from langchain.smith import RunEvalConfig

evaluation_config = RunEvalConfig(
    # Evaluators can either be an evaluator type (e.g., "qa", "criteria", "embedding_distance", etc.) or a configuration for that evaluator
    evaluators=[
        check_not_idk,
        # Measures whether a QA response is "Correct", based on a reference answer
        # You can also select via the raw string "qa"
        EvaluatorType.QA,
        # Measure the embedding distance between the output and the reference answer
        # Equivalent to: EvalConfig.EmbeddingDistance(embeddings=OpenAIEmbeddings())
        EvaluatorType.EMBEDDING_DISTANCE,
        # Grade whether the output satisfies the stated criteria.
        # You can select a default one such as "helpfulness" or provide your own.
        RunEvalConfig.LabeledCriteria("helpfulness"),
        # The LabeledScoreString evaluator outputs a score on a scale from 1-10.
        # You can use default criteria or write our own rubric
        RunEvalConfig.LabeledScoreString(
            {
                "accuracy": """
Score 1: The answer is completely unrelated to the reference.
Score 3: The answer has minor relevance but does not align with the reference.
Score 5: The answer has moderate relevance but contains inaccuracies.
Score 7: The answer aligns with the reference but has minor errors or omissions.
Score 10: The answer is completely accurate and aligns perfectly with the reference."""
            },
            normalize_by=10,
        ),
    ],
    batch_evaluators=[max_pred_length],
)

### 4. Run the agent and evaluators

Use the [run_on_dataset](https://api.python.langchain.com/en/latest/smith/langchain.smith.evaluation.runner_utils.run_on_dataset.html#langchain.smith.evaluation.runner_utils.run_on_dataset) (or asynchronous [arun_on_dataset](https://api.python.langchain.com/en/latest/smith/langchain.smith.evaluation.runner_utils.arun_on_dataset.html#langchain.smith.evaluation.runner_utils.arun_on_dataset)) function to evaluate your model. This will:
1. Fetch example rows from the specified dataset.
2. Run your agent (or any custom function) on each example.
3. Apply evaluators to the resulting run traces and corresponding reference examples to generate automated feedback.

The results will be visible in the LangSmith app.

In [13]:
from langchain import hub

# We will test this version of the prompt
prompt = hub.pull("wfh/langsmith-agent-prompt:798e7324")

In [14]:
import functools

from langchain.smith import arun_on_dataset, run_on_dataset

chain_results = run_on_dataset(
    dataset_name=dataset_name,
    llm_or_chain_factory=functools.partial(
        create_agent, prompt=prompt, llm_with_tools=llm_with_tools
    ),
    evaluation=evaluation_config,
    verbose=True,
    client=client,
    project_name=f"tools-agent-test-5d466cbc-{unique_id}",
    # Project metadata communicates the experiment parameters,
    # Useful for reviewing the test results
    project_metadata={
        "env": "testing-notebook",
        "model": "gpt-3.5-turbo",
        "prompt": "5d466cbc",
    },
)

# Sometimes, the agent will error due to parsing issues, incompatible tool inputs, etc.
# These are logged as warnings here and captured as errors in the tracing UI.

View the evaluation results for project 'tools-agent-test-5d466cbc-ac81f626' at:
https://smith.langchain.com/o/5bb51f22-2934-4e47-aafe-41a1950edaa5/datasets/f432944d-0aed-4bca-be5e-9290802e2765/compare?selectedSessions=416f5722-7af8-4a0e-9726-a0804aaeee12

View all tests for Dataset agent-qa-ac81f626 at:
https://smith.langchain.com/o/5bb51f22-2934-4e47-aafe-41a1950edaa5/datasets/f432944d-0aed-4bca-be5e-9290802e2765
[>                                                 ] 0/5

/Users/harrywang/sandbox/ai-tutorials/venv/lib/python3.10/site-packages/langchain_core/_api/deprecation.py:117: LangChainDeprecationWarning: The class `langchain_community.embeddings.openai.OpenAIEmbeddings` was deprecated in langchain-community 0.0.9 and will be removed in 0.2.0. An updated version of the class exists in the langchain-openai package and should be used instead. To use it run `pip install -U langchain-openai` and import as `from langchain_openai import OpenAIEmbeddings`.
  warn_deprecated(


[------------------------------------------------->] 5/5

,feedback.not_uncertain,feedback.correctness,feedback.embedding_cosine_distance,feedback.helpfulness,feedback.score_string:accuracy,error,execution_time,run_id
count,5.0,5.000000,5.000000,5.000000,5.000000,0,5.000000,5
unique,NaN,NaN,NaN,NaN,NaN,0,NaN,5
top,NaN,NaN,NaN,NaN,NaN,NaN,NaN,355b5d20-467d-4482-945e-dbb25d3c33f6
freq,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1
mean,1.0,0.600000,0.130842,0.800000,0.780000,NaN,3.564310,NaN
std,0.0,0.547723,0.099893,0.447214,0.216795,NaN,0.809289,NaN
min,1.0,0.000000,0.039426,0.000000,0.500000,NaN,2.643885,NaN
25%,1.0,0.000000,0.066036,1.000000,0.700000,NaN,3.136887,NaN
50%,1.0,1.000000,0.098833,1.000000,0.700000,NaN,3.317314,NaN
75%,1.0,1.000000,0.159314,1.000000,1.000000,NaN,4.003156,NaN


### Review the test results

You can review the test results tracing UI below by clicking the URL in the output above or navigating to the "Testing & Datasets" page in LangSmith  **"agent-qa-{unique_id}"** dataset.

![test results](https://github.com/langchain-ai/langchain/blob/master/docs/docs/langsmith/img/test_results.png?raw=1)

This will show the new runs and the feedback logged from the selected evaluators. You can also explore a summary of the results in tabular format below.

In [15]:
chain_results.to_dataframe()

,inputs.input,outputs.input,outputs.output,reference.output,feedback.not_uncertain,feedback.correctness,feedback.embedding_cosine_distance,feedback.helpfulness,feedback.score_string:accuracy,error,execution_time,run_id
5d1bbe42-5138-448a-bca9-958ae84a3c51,What is the langsmith cookbook?,What is the langsmith cookbook?,The LangSmith Cookbook is a repository that co...,The langsmith cookbook is a github repository ...,1,1,0.039426,1,1.0,None,4.003156,355b5d20-467d-4482-945e-dbb25d3c33f6
a6e3a2f4-5d5f-463b-a5db-993dfb08f8f6,When did langchain first announce the hub?,When did langchain first announce the hub?,LangChain first announced the LangChain Hub a ...,"September 5, 2023",1,0,0.290599,0,0.5,None,3.136887,916a69a2-b180-43b5-98df-36bcf575a127
cd6424db-09ba-4a17-8d71-bfa3a3ffd98d,What is LangChain?,What is LangChain?,LangChain is an open-source Python library tha...,LangChain is an open-source framework for buil...,1,0,0.098833,1,0.7,None,4.720308,7bd28536-1f51-4d88-ab5a-89bafe9a93c7
c73af836-bafb-4d67-b961-81b997cfa243,What's LangSmith?,What's LangSmith?,LangSmith is a platform that allows you to tes...,"LangSmith is a unified platform for debugging,...",1,1,0.066036,1,0.7,None,3.317314,1000ee5d-8808-48e2-855c-a70ebd97b4fb
e32104e9-8c05-46cf-9823-ac5cffe36731,When was Llama-v2 released?,When was Llama-v2 released?,"LangChain Llama-v2 was released on July 18, 2023.","July 18, 2023",1,1,0.159314,1,1.0,None,2.643885,47333bf5-d9ce-4fcd-81f1-06cb6d68afd4


### (Optional) Compare to another prompt

Now that we have our test run results, we can make changes to our agent and benchmark them. Let's try this again with a different prompt and see the results.

In [16]:
candidate_prompt = hub.pull("wfh/langsmith-agent-prompt:39f3bbd0")

chain_results = run_on_dataset(
    dataset_name=dataset_name,
    llm_or_chain_factory=functools.partial(
        create_agent, prompt=candidate_prompt, llm_with_tools=llm_with_tools
    ),
    evaluation=evaluation_config,
    verbose=True,
    client=client,
    project_name=f"tools-agent-test-39f3bbd0-{unique_id}",
    project_metadata={
        "env": "testing-notebook",
        "model": "gpt-3.5-turbo",
        "prompt": "39f3bbd0",
    },
)

View the evaluation results for project 'tools-agent-test-39f3bbd0-ac81f626' at:
https://smith.langchain.com/o/5bb51f22-2934-4e47-aafe-41a1950edaa5/datasets/f432944d-0aed-4bca-be5e-9290802e2765/compare?selectedSessions=8c1f4ec1-78a6-4c36-b0a5-a33ee1bfdba8

View all tests for Dataset agent-qa-ac81f626 at:
https://smith.langchain.com/o/5bb51f22-2934-4e47-aafe-41a1950edaa5/datasets/f432944d-0aed-4bca-be5e-9290802e2765
[------------------------------------------------->] 5/5

,feedback.not_uncertain,feedback.correctness,feedback.embedding_cosine_distance,feedback.helpfulness,feedback.score_string:accuracy,error,execution_time,run_id
count,5.0,5.0,5.000000,5.0,5.000000,0,5.000000,5
unique,NaN,NaN,NaN,NaN,NaN,0,NaN,5
top,NaN,NaN,NaN,NaN,NaN,NaN,NaN,f3e630c8-a48b-473d-ac63-2d0261d1f0de
freq,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1
mean,1.0,1.0,0.104468,1.0,0.880000,NaN,3.457657,NaN
std,0.0,0.0,0.053067,0.0,0.164317,NaN,1.369182,NaN
min,1.0,1.0,0.041164,1.0,0.700000,NaN,2.319222,NaN
25%,1.0,1.0,0.067801,1.0,0.700000,NaN,2.472358,NaN
50%,1.0,1.0,0.096737,1.0,1.000000,NaN,3.287395,NaN
75%,1.0,1.0,0.153193,1.0,1.000000,NaN,3.470725,NaN


## Exporting datasets and runs

LangSmith lets you export data to common formats such as CSV or JSONL directly in the web app. You can also use the client to fetch runs for further analysis, to store in your own database, or to share with others. Let's fetch the run traces from the evaluation run.

**Note: It may be a few moments before all the runs are accessible.**

In [17]:
runs = client.list_runs(project_name=chain_results["project_name"], execution_order=1)

In [18]:
# The resulting tests are stored in a project.  You can programmatically
# access important metadata from the test, such as the dataset version it was run on
# or your application's revision ID.
client.read_project(project_name=chain_results["project_name"]).metadata

{'env': 'testing-notebook',
 'git': {'tags': None,
  'dirty': True,
  'branch': 'main',
  'commit': 'befa9af80d9f4362ec789e2cece964c695f9ce89',
  'remote_url': 'https://github.com/harrywang/langchain-tutorials.git',
  'author_name': 'Harry Wang',
  'commit_time': '1711650818',
  'author_email': 'harryjwang@gmail.com'},
 'model': 'gpt-3.5-turbo',
 'prompt': '39f3bbd0',
 'dataset_version': '2024-04-02T02:49:18.716384+00:00'}

In [19]:
# After some time, the test metrics will be populated as well.
client.read_project(project_name=chain_results["project_name"]).feedback_stats

{'correctness': {'n': 5, 'avg': 1.0},
 'embedding_cosine_distance': {'n': 5, 'avg': 0.10442},
 'helpfulness': {'n': 5, 'avg': 1.0},
 'not_uncertain': {'n': 5, 'avg': 1.0},
 'score_string:accuracy': {'n': 5, 'avg': 0.8800000000000001}}

## Conclusion

Congratulations! You have successfully traced and evaluated an agent using LangSmith!

This was a quick guide to get started, but there are many more ways to use LangSmith to speed up your developer flow and produce better results.

For more information on how you can get the most out of LangSmith, check out [LangSmith documentation](https://docs.smith.langchain.com/), and please reach out with questions, feature requests, or feedback at [support@langchain.dev](mailto:support@langchain.dev).